In [1]:
"""
#Loads a CSV DwC occurrence file that has been augmented with BELS locality strings
"""

'\n#Loads a CSV DwC occurrence file that has been augmented with BELS locality strings\n'

In [41]:
import pandas as pd
import matplotlib

# switch to suppress CSV writing (which is slow)
write_csv = True

In [2]:
#df_occ = pd.read_csv('TORCH-Denton-9f30f07b-e694-4f79-83c3-54fafaa98727.csv', low_memory=False)
#df_occ = pd.read_csv('r_dale_thomas_texas_bels.csv', low_memory=False)
#df_occ = pd.read_csv('r-dale-thomas-all-seinet-texas-bels.csv', low_memory=False)
df_occ = pd.read_csv('torch_bels_locs.csv', low_memory=False, sep='\t')

In [4]:
df_occ.shape

(1358428, 126)

In [57]:
# Drop irrelevant colums
drop_columns= ['higherClassification','kingdom','phylum','class','order','identificationReferences','identificationRemarks','taxonRemarks','identificationQualifier','typeStatus','fieldNumber','eventID','informationWithheld','dataGeneralizations','dynamicProperties','associatedSequences','associatedTaxa','reproductiveCondition','establishmentMeans','lifeStage','sex','individualCount','samplingProtocol','preparations','continent','waterBody','islandGroup','island','rights','rightsHolder','accessRights','recordID','type','license','bibliographicCitation','datasetName','fieldNotes','countryCode','nomenclaturalCode','nomenclaturalStatus','associatedMedia','higherGeography','institutionID','georeferencedDate','datasetID','occurrenceStatus','verbatimLocality','organismID','previousIdentifications','eventTime','eventRemarks','locationAccordingTo','verbatimCoordinateSystem','footprintWKT','earliestEonOrLowestEonothem','earliestEraOrLowestErathem','earliestPeriodOrLowestSystem','earliestEpochOrLowestSeries','earliestAgeOrLowestStage','group','formation','member','identificationVerificationStatus','scientificNameID']
df_occ = df_occ.drop(columns=drop_columns)

In [58]:
df_occ.shape

(1358428, 62)

In [5]:
#list(df_occ.columns)

In [59]:
# Find duplicate records

#bels_matchwithcoords
df_matches = df_occ[df_occ['bels_location_string'].duplicated(keep=False)]
#df_matches = df_denton[df_denton.bels_matchwithcoords

In [60]:
df_matches.shape

(957794, 62)

In [61]:
#list(df_matches.columns)

In [62]:
# Add location ID
# first create a copy to avoid copy of a slice of a DF
df_matches = df_matches.copy()
# https://stackoverflow.com/a/51110197 or https://stackoverflow.com/a/51110205
df_matches['loc_id'] = df_matches.groupby(['bels_location_string']).ngroup()

In [63]:
# Add dup loc count column
#https://stackoverflow.com/a/46768694

df_matches['dup_loc_count'] = df_matches.groupby(['bels_location_string']).transform('size')

In [64]:
#df_matches

In [65]:
if write_csv:
    df_matches.to_csv('TORCH_bels_matches_loc_id.csv')

In [67]:
# find records lacking geo
#bels_decimallatitude
df_nogeo = df_matches[df_matches['decimalLatitude'].isna()]
#TODO add loc ID - https://stackoverflow.com/a/51110205


In [68]:
df_nogeo.shape

(631546, 64)

In [69]:
if write_csv:
    df_nogeo.to_csv('TORCH_bels_matches_nogeo.csv')

In [70]:
# find records with geo
df_geo = df_matches[df_matches['decimalLatitude'].notna()]

In [71]:
df_geo.shape

(326248, 64)

In [72]:
if write_csv:
    df_geo.to_csv('TORCH_bels_matches_geo.csv')

In [73]:
# all dups with and without geocoords
dup_loc_count = df_matches.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [74]:
dup_loc_count.shape

(144912,)

In [75]:
if write_csv:
    dup_loc_count.to_csv('TORCH_dup_loc_count.csv')

In [76]:
# dups with no goecoords
#dups = df.pivot_table(index = ['Course'], aggfunc ='size') 
dup_loc_count_no_geo = df_nogeo.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [77]:
dup_loc_count_no_geo.shape

(96583,)

In [78]:
if write_csv:
    dup_loc_count_no_geo.to_csv('TORCH_dup_loc_count_nogeo.csv')

In [79]:
# dups with goecoords
dup_loc_count_geo = df_geo.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [80]:
dup_loc_count_geo.shape

(64953,)

In [81]:
if write_csv:
    dup_loc_count_geo.to_csv('TORCH_dup_loc_count_geo.csv')

In [82]:
#print(dup_loc_count)

In [83]:
#filter records that mention no additional locality?

In [84]:
#list(df_matches.columns)

In [85]:
# from ChatGPT
#https://chat.openai.com/share/21c561bb-db43-47ee-980a-f88078b8c78b
# Identify duplicate values in a specific column
#
#duplicates = df_matches['bels_matchwithcoords'].duplicated(keep=False)
#duplicates = df_matches['bels_location_string'].duplicated(keep=False)

# Count the number of duplicates for each value
#duplicate_counts = duplicates.groupby(df_matches['bels_matchwithcoords']).sum()
#duplicate_counts = duplicates.groupby(df_matches['bels_location_string']).sum()



In [86]:
# Note needed, done above
# Add a new column to the DataFrame with the number of duplicates
#df_matches['duplicate_count'] = df_matches['bels_location_string'].map(duplicate_counts)

In [87]:
#duplicate_counts.sort_index().cumsum().plot()
#duplicate_counts.set_index('duplicate_count')
#duplicate_counts.sort_index().plot()

In [88]:
#df_matches[['duplicate_count']] = df_matches[['duplicate_count']].apply(pd.to_numeric)

In [89]:
#df_matches.set_index('duplicate_count').cumsum()

In [90]:
#TODO
#Load ids of elite georeferencers - EG
df_georeferencers = pd.read_csv('TORCH-georeferencers_tested.csv', low_memory=False)

#find records with geo that have been done by EGs
#find records without geo that match those done by EGs

In [91]:
# filter records that have been georeferenced by 'elite' vetted georeferencers
df_by_egeo = df_matches[df_matches['georeferencedBy'].isin(df_georeferencers['Username'])]

In [92]:
df_by_egeo.shape

(9060, 64)

In [93]:
if write_csv:
    df_by_egeo.to_csv('TORCH_df_by_egeo.csv')

In [94]:
# find locations that match loc_id of those done by EG
df_loc_match_by_egeo = df_matches[df_matches['loc_id'].isin(df_by_egeo['loc_id'])]

In [95]:
df_loc_match_by_egeo.shape

(20543, 64)

In [96]:
# find matching locs that have not been georeffed
df_loc_match_by_egeo_nogeo = df_loc_match_by_egeo[df_loc_match_by_egeo['decimalLatitude'].isna()]

In [97]:
df_loc_match_by_egeo_nogeo.shape

(8066, 64)

In [98]:
# find matching locs that have been georeffed
df_loc_match_by_egeo_w_geo = df_loc_match_by_egeo[df_loc_match_by_egeo['decimalLatitude'].notna()]

In [99]:
df_loc_match_by_egeo_w_geo.shape

(12477, 64)